# Parameter Optimization with Optuna

In this example we will train a RandomForest model and optimize its parameters using [Optuna](https://optuna.readthedocs.io/en/stable/).
This example is an adapted version from the Optuna [Basic Concept example](https://optuna.readthedocs.io/en/stable/#basic-concepts).



In [5]:
# Setup temporary directory and initialize git and dvc
from zntrack import config

config.nb_name = "parameter_optimization.ipynb"

from zntrack.utils import cwd_temp_dir

temp_dir = cwd_temp_dir()

!git init
!dvc init

Initialized empty Git repository in /tmp/tmphi4dsmja/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


## Workflow
Our Workflow consists of multiple steps:
- Download the dataset
- Split into train / test data
- Train a RandomForest model on the train data
- Evaluate the model on the test data

We want to optimize using two differen Models: RandomForest and LinearSVR with their respective hyperparameters.
We want to optimize them and use the `Evaluate` Node to compute a RMSE that Optuna optimizes.
We will use DVC [Experiments](https://dvc.org/doc/start/experiments) to track each run.
In combination with Optuna, this allows us not only to optimize the parameters but also easily store and access the trained models afterwards.


[![](https://mermaid.ink/img/pako:eNp1j7sOgkAQRX-FTC0FYEVhhYmNFXSuxQQG2GQfZJnVGMK_OzFKaKxmcu6981ig9R1BCb3xz3bEwElTKafYCc5uCs4PNBGZFNx_OBd88XHWbqiQsSbeiYWIV6lmx47CmoDaNTRzPRm9D-RpevqYtglfkG3xf6BQDg5gKVjUnfywKJckCngkK_eW0nbUYzSyTrlVrBjZ1y_XQskh0gHi1MlrlcYhoIWyRzPT-gaiDmCv?type=png)](https://mermaid.live/edit#pako:eNp1j7sOgkAQRX-FTC0FYEVhhYmNFXSuxQQG2GQfZJnVGMK_OzFKaKxmcu6981ig9R1BCb3xz3bEwElTKafYCc5uCs4PNBGZFNx_OBd88XHWbqiQsSbeiYWIV6lmx47CmoDaNTRzPRm9D-RpevqYtglfkG3xf6BQDg5gKVjUnfywKJckCngkK_eW0nbUYzSyTrlVrBjZ1y_XQskh0gHi1MlrlcYhoIWyRzPT-gaiDmCv)

In [6]:
import optuna, sklearn, zntrack
import sklearn.datasets
import sklearn.ensemble
import sklearn.model_selection
import sklearn.metrics


class HousingDataSet(zntrack.Node):
    """Download and prepare the California housing dataset."""

    data = zntrack.dvc.outs("scikit_learn_data")

    def run(self) -> None:
        _ = sklearn.datasets.fetch_california_housing(
            data_home=self.data, return_X_y=True
        )

    @property
    def labels(self) -> dict:
        _, labels = sklearn.datasets.fetch_california_housing(
            data_home=self.data, return_X_y=True
        )
        return labels

    @property
    def features(self) -> dict:
        features, _ = sklearn.datasets.fetch_california_housing(
            data_home=self.data, return_X_y=True
        )
        return features


class TrainTestSplit(zntrack.Node):
    """Split the dataset into train and test sets."""

    labels = zntrack.zn.deps()
    features = zntrack.zn.deps()
    seed = zntrack.zn.params(1234)

    train_features = zntrack.zn.outs()
    test_features = zntrack.zn.outs()
    train_labels = zntrack.zn.outs()
    test_labels = zntrack.zn.outs()

    def run(self) -> None:
        self.train_features, self.test_features, self.train_labels, self.test_labels = (
            sklearn.model_selection.train_test_split(
                self.features, self.labels, test_size=0.2, random_state=self.seed
            )
        )


class RandomForest(zntrack.Node):
    """Train a random forest model."""

    train_features = zntrack.zn.deps()
    train_labels = zntrack.zn.deps()
    seed = zntrack.zn.params(1234)
    max_depth = zntrack.zn.params()

    model = zntrack.zn.outs()

    def run(self) -> None:
        self.model = sklearn.ensemble.RandomForestRegressor(
            random_state=self.seed, max_depth=self.max_depth
        )
        self.model.fit(self.train_features, self.train_labels)


class LinearSVR(zntrack.Node):
    """Train a SVR model."""

    train_features = zntrack.zn.deps()
    train_labels = zntrack.zn.deps()
    C = zntrack.zn.params()

    model = zntrack.zn.outs()

    def run(self) -> None:
        self.model = sklearn.svm.LinearSVR(C=self.C)
        self.model.fit(self.train_features, self.train_labels)


class Evaluate(zntrack.Node):
    """Evaluate the model on a test set."""

    model = zntrack.zn.deps()
    test_features = zntrack.zn.deps()
    test_labels = zntrack.zn.deps()

    score = zntrack.zn.metrics()

    def run(self) -> None:
        prediction = self.model.predict(self.test_features)
        self.score = sklearn.metrics.mean_squared_error(self.test_labels, prediction)

We use the `zntrack.Project` to create our workflow as usual.
To use DVC Experiments, we need to create an initial commit.
Therefore, we run the project directly and make an initial git commit afterwards.

In [7]:
with zntrack.Project() as project:
    data = HousingDataSet()
    split = TrainTestSplit(labels=data.labels, features=data.features)
    model = RandomForest(
        train_features=split.train_features,
        train_labels=split.train_labels,
        max_depth=2,
        name="model",
    )
    evaluate = Evaluate(
        model=model.model,
        test_features=split.test_features,
        test_labels=split.test_labels,
    )

project.run()

Running DVC command: 'stage add --name HousingDataSet --force ...'
Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.
 

 

Running DVC command: 'stage add --name TrainTestSplit --force ...'
 

 

Running DVC command: 'stage add --name model --force ...'
 

 

Running DVC command: 'stage add --name Evaluate --force ...'
 

 

Running DVC command: 'repro'


In [8]:
RandomForest.from_rev(name="model").state

 

 

NodeStatus(loaded=True, results=<NodeStatusResults.UNKNOWN: 0>, remote=None, rev=None)

In [9]:
!git add .
!git commit -m "initial commit"

[main (root-commit) b5218c7] initial commit
 20 files changed, 2037 insertions(+)
 create mode 100644 .dvc/.gitignore
 create mode 100644 .dvc/config
 create mode 100644 .dvcignore
 create mode 100644 .gitignore
 create mode 100644 dvc.lock
 create mode 100644 dvc.yaml
 create mode 100644 nodes/Evaluate/score.json
 create mode 100644 nodes/TrainTestSplit/.gitignore
 create mode 100644 nodes/model/.gitignore
 create mode 100644 parameter_optimization.ipynb
 create mode 100644 params.yaml
 create mode 100644 src/Evaluate.py
 create mode 100644 src/HousingDataSet.py
 create mode 100644 src/RandomForest.py
 create mode 100644 src/TrainTestSplit.py
 create mode 100644 src/__pycache__/Evaluate.cpython-310.pyc
 create mode 100644 src/__pycache__/HousingDataSet.cpython-310.pyc
 create mode 100644 src/__pycache__/RandomForest.cpython-310.pyc
 create mode 100644 src/__pycache__/TrainTestSplit.cpython-310.pyc
 create mode 100644 zntrack.json


## Optimize

For Optuna we need to define an objective we want to optimize.
We use the `project.create_experiment` API from ZnTrack to change the model parameter and return the score from the `Evaluate` stage as final metric to optimize.
To later identify the experiments, we name them according to the `trial.number` from optuna.

In [10]:
def objective(trial):
    with project.create_experiment(queue=False, name=f"exp-{trial.number}") as exp:
        regressor_name = trial.suggest_categorical("classifier", ["SVR", "RandomForest"])

        # we need to replace the existing model on the graph with a new model.

        project.remove("model")

        if regressor_name == "SVR":
            svr_c = trial.suggest_float("svr_c", 1e-10, 1e10, log=True)
            model = LinearSVR(
                train_features=split.train_features,
                train_labels=split.train_labels,
                C=svr_c,
                name="model",
            )
        else:
            max_depth = trial.suggest_int("max_depth", 2, 32)
            model = RandomForest(
                train_features=split.train_features,
                train_labels=split.train_labels,
                max_depth=max_depth,
                name="model",
            )

        # need to let the evaluate node know which model to evaluate
        evaluate.model = model.model

    return exp[evaluate].score


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

[I 2023-05-30 14:28:03,838] A new study created in memory with name: no-name-15e1292e-d549-44d4-9bf0-c84394860091
Running DVC command: 'stage add --name HousingDataSet --force ...'
 

 

Running DVC command: 'stage add --name TrainTestSplit --force ...'
 

 

Running DVC command: 'stage add --name model --force ...'
 

...

 

Running DVC command: 'stage add --name Evaluate --force ...'
 

 

Running DVC command: 'exp apply exp-9'
 [I 2023-05-30 14:30:24,324] Trial 9 finished with value: 3.1497504942833037 and parameters: {'classifier': 'SVR', 'svr_c': 275021882.81023085}. Best is trial 4 with value: 11.490470659940513.


 

## Evaluate

We can now investigate the best parameters via `study.best_params`.
Additionally, because we used DVC experiments we can directly access the experiment with the best parameters, by the name we used.

In [11]:
study.best_params

{'classifier': 'SVR', 'svr_c': 166.57184086434526}

In [12]:
project.experiments.keys()

dict_keys([None, 'exp-9', 'exp-8', 'exp-7', 'exp-6', 'exp-5', 'exp-4', 'exp-3', 'exp-2', 'exp-1', 'exp-0'])

We can either load the Node via the experiment or by it's name using `zntrack.from_rev()`.
The node should not be loaded via `model.load()` because the `model` instance could be `RandomForest` and the best model would be `LinearSVR` or *vice versa*.

In [16]:
exp = project.experiments[f"exp-{study.best_trial.number}"]
best_model = exp["model"]
best_model = zntrack.from_rev("model", rev=f"exp-{study.best_trial.number}")

ModuleNotFoundError: No module named 'src'. The package might be available via 'pip install src' or from the remote via 'pip install git+.'.

In [17]:
f"exp-{study.best_trial.number}"

'exp-4'

In [14]:
import zntrack

In [18]:
zntrack.from_rev("model", rev=f"exp-4")

ModuleNotFoundError: No module named 'src'. The package might be available via 'pip install src' or from the remote via 'pip install git+.'.

In [11]:
# we load split data into memory to compute the score.
split.load()

best_score = evaluate.from_rev(rev=f"exp-{study.best_trial.number}").score
initial_score = evaluate.from_rev(rev="HEAD").score
print(f"Best score: {best_score:.3f} compared to initial score: {initial_score:.3f}")

Best score: 9.748 compared to initial score: 0.750


temp_dir.cleanup()